In [ ]:
import json
import os
import fnmatch
import pandas as pd
import re
import numpy as np

In [2]:
####### example to access text of json file


s1 = pd.Series([0,'test', 0])
df = pd.DataFrame([list(s1)], columns=['id', 'text','label'])
index = 0

for path, dirs, files in os.walk('twitter-english/'):
    if (path.endswith('source-tweet') or path.endswith('replies')):
        for filename in os.listdir(path):
            if filename.endswith('.json'):
                with open(os.path.join(path,filename)) as jsonFile:
                    jsonSourceTweet = json.load(jsonFile)

                    text_tweet = jsonSourceTweet['text']
                    id_tweet = int(re.sub("[^0-9]", "", filename))

                    newline = [id_tweet, text_tweet, index]
                    df.loc[index] = newline

                    index += 1




In [3]:
df.head()

id                                               text  \
0  529661510369284096  @mikeyerxa EXCUSE ME, WHERE'S MY INVITE (to th...   
1  529661727940431872  @halihamilton @mikeyerxa gonna try to make it ...   
2  529661317087768576  @mikeyerxa you should send Prince a set of ten...   
3  529666750330449920  @NobleRobel Girl, don't be mean.   Age ain't n...   
4  529660296080916480  Prince is playing a secret show in Toronto ton...   

   label  
0      0  
1      1  
2      2  
3      3  
4      4

In [4]:
df_label = pd.read_json('train-key.json')

In [5]:
df_label.columns = ['label', 'b']
df_label

label      b
501760642928635904  comment    NaN
500270212198174720  comment    NaN
524971210275565568  comment    NaN
552836882770690049  comment    NaN
500289931097296897  support    NaN
...                     ...    ...
8gp0d7                query   true
8i274x                query  false
8n9173                query  false
8unvgg                query  false
8yktu5                 deny  false

[5217 rows x 2 columns]

In [6]:
for i in range(len(df)):
    try:
        index = df.loc[i].id
        df.loc[i, 'label'] = df_label.loc[str(index)].label
    except KeyError:
        df.loc[i, 'label'] = 'not_found'


In [7]:
df.label.value_counts()

comment      2907
not_found    1049
support       910
query         358
deny          344
Name: label, dtype: int64

In [8]:
len(df.label.unique())

5

In [9]:

#get rid of not found tag

index_to_delete = df[df.label == "not_found"].index
mask = np.logical_not(df.index.isin(index_to_delete))

df_final = df[mask]
df_final.label.value_counts()


comment    2907
support     910
query       358
deny        344
Name: label, dtype: int64

In [10]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = list(df.text)
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)


ModuleNotFoundError: No module named 'transformers'

In [21]:
def create_test_data(path_training, path_answer):
    s1 = pd.Series([0,'test', 0])
    df = pd.DataFrame([list(s1)], columns=['id', 'text','label'])
    index = 0

    for path, dirs, files in os.walk(path_training):
        if (path.endswith('source-tweet') or path.endswith('replies')):
            for filename in os.listdir(path):
                if filename.endswith('.json'):
                    with open(os.path.join(path,filename)) as jsonFile:
                        jsonSourceTweet = json.load(jsonFile)

                        text_tweet = jsonSourceTweet['text']
                        id_tweet = int(re.sub("[^0-9]", "", filename))

                        newline = [id_tweet, text_tweet, index]
                        df.loc[index] = newline

                        index += 1

    #rename column into label                    
    df_label = pd.read_json(path_answer)
    first_col = df_label.columns[0]
    df_label.rename(columns={first_col:'label'}, inplace=True)
    
    #add labels
    for i in range(len(df)):
        try:
            index = df.loc[i].id
            df.loc[i, 'label'] = df_label.loc[str(index)].label
        except KeyError:
            df.loc[i, 'label'] = 'not_found'

    #delete unecessary labels
    index_to_delete = df[df.label == "not_found"].index
    mask = np.logical_not(df.index.isin(index_to_delete))
    df_final = df[mask]


    return df_final

tes_data_csv = create_test_data('twitter-en-test-data', 'final-eval-key.json')


In [24]:
tes_data_csv.label.value_counts()

comment    771
support    141
deny        92
query       62
Name: label, dtype: int64